# Module Imports

In [4]:
import os

import pandas as pd

# Loading the data

##### Using OS module to determine filepaths in OS independent manner as backslashes used on Windows and forward slashes used on Unix

In [5]:
working_directory = os.path.abspath('')

In [7]:
training_data_location = os.path.join(working_directory, 'Data', 'test.csv')

In [ ]:
titanic_data = pd.DataFrame.from_csv()